In [1]:
from langchain.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper
import yfinance
import json


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}


functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

In [2]:
import openai as client
openai.api_key = 
# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     model="gpt-4-1106-preview",
#     tools=functions,
# )
assistant_id = "asst_LjyGnlJW0d5roo2O1Qi6rBnw"

In [3]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_Z8cULteKBQXFyLDNoVrjSblL', created_at=1713104295, metadata={}, object='thread')

In [4]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_o1FRUtPRUJzWtPrn3dF8I9Nb', assistant_id='asst_LjyGnlJW0d5roo2O1Qi6rBnw', cancelled_at=None, completed_at=None, created_at=1713104295, expires_at=1713104895, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_Z8cULteKBQXFyLDNoVrjSblL', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'type':

In [5]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )

def get_required_action(run_id, thread_id):
    run = get_run(run_id, thread_id)
    return run.required_action

def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

# def get_messages(thread_id):
#     messages = client.beta.threads.messages.list(thread_id=thread_id).data
#     messages.reverse()
#     for message in messages:
#         print(f"{message.role}: ", end=' ')
#     if message.content:
#         print(f"{message.content[0].text.value}")     

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread.id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        print(json.loads(function.arguments))
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id, thread_id=thread_id, tool_outputs=outpus
    )

In [7]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_ticker with arg {"company_name":"Salesforce"}
{'company_name': 'Salesforce'}


Run(id='run_o1FRUtPRUJzWtPrn3dF8I9Nb', assistant_id='asst_LjyGnlJW0d5roo2O1Qi6rBnw', cancelled_at=None, completed_at=None, created_at=1713104295, expires_at=1713104895, failed_at=None, file_ids=[], instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1713104296, status='queued', thread_id='thread_Z8cULteKBQXFyLDNoVrjSblL', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_statement', description="Given a ticker symbol (i.e AAPL) returns the company's income statement.", parameters={'

In [ ]:
print(get_run(run.id, thread.id).status,get_required_action(run.id, thread.id))

requires_action RequiredAction(submit_tool_outputs=RequiredActionSubmitToolOutputs(tool_calls=[RequiredActionFunctionToolCall(id='call_jRCojIwxn3djHXUatlLEDjCi', function=Function(arguments='{"company_name": "Salesforce"}', name='get_ticker'), type='function'), RequiredActionFunctionToolCall(id='call_77Kp2VKqjrShlQyrKuoaNHBb', function=Function(arguments='{"ticker": "CRM"}', name='get_income_statement'), type='function'), RequiredActionFunctionToolCall(id='call_8uWR45mNp7lQxS4GSvHRDelV', function=Function(arguments='{"ticker": "CRM"}', name='get_balance_sheet'), type='function'), RequiredActionFunctionToolCall(id='call_FZhbPkOHA4sHY74rv5qTcy1S', function=Function(arguments='{"ticker": "CRM"}', name='get_daily_stock_performance'), type='function')]), type='submit_tool_outputs')


In [ ]:
get_messages(thread.id)

user:  assistant:  assistant:  The evaluation of Salesforce (CRM) stock as a potential investment would require a careful analysis of the company's financial performance, along with market trends and other factors that may influence the stock value. Based on the data provided, an in-depth look at Salesforce's most recent financial statements and stock performance takes into account several key areas.

Financials:

**Income Statement Summary**
- **Most Recent Quarter (MRQ):**
  - Total Revenue: $34.86B
  - Gross Profit: $26.32B
  - Operating Income: $5.00B
  - Net Income: $4.14B
  - EBITDA: $9.96B
  - EPS (Diluted): $4.20

- **Year-over-Year (YoY) Comparison to Previous MRQ:**
  - Total Revenue: Increased from $31.35B
  - Net Income: Increased significantly from $208M
  - EBITDA: Increased from $5.64B

This recent performance indicates growth in both top-line revenue and bottom-line profit, suggesting an improvement in the company's financial health.

**Balance Sheet Summary**
- **MRQ:*

In [ ]:
send_message(thread.id, "go ahead")

BadRequestError: Error code: 400 - {'error': {'message': "Can't add messages to thread_40IU5K0qjBjOuNDQKEjWVDLk while a run run_kB3L5vZ61dA5jXSp4y9CCXmH is active.", 'type': 'invalid_request_error', 'param': None, 'code': None}}